<a href="https://colab.research.google.com/github/fmahboub/Spam_Classifier_Naive_Bayes/blob/master/Spam_Naive_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

In [0]:
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam (1).csv


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['spam.csv']), encoding = "ISO-8859-1")

In [0]:
df = df.drop("Unnamed: 2", axis=1)
df = df.drop("Unnamed: 3", axis=1)
df = df.drop("Unnamed: 4", axis=1)
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [0]:
ham = []
i = 0
for x in df['v1']:
  if x == 'ham':
    ham.append(df['v2'].iloc[i])
  i +=1

In [0]:
spam = []
i = 0
for x in df['v1']:
  if x == 'spam':
    spam.append(df['v2'].iloc[i])
  i += 1

In [0]:
ham_split = []
for x in ham:
    ham_split.append((x.split(' '),'ham'))

In [0]:
spam_split = []
for x in spam:
    spam_split.append((x.split(' '),'spam'))

In [0]:
#Merge words from all SMS
full_SMS_split = ham_split + spam_split 
train = int((4/5)*len(full_SMS_split))

In [0]:
from nltk.sentiment import SentimentAnalyzer
import nltk.sentiment.util
from nltk.classify import NaiveBayesClassifier

#Train/Test split of review words
random.shuffle(full_SMS_split)
training_docs = full_SMS_split[:train]
test_docs = full_SMS_split[train:]

print("Training: %d, Testing: %d" % (len(training_docs), len(test_docs)))

sentim_analyzer = SentimentAnalyzer()

Training: 4457, Testing: 1115


In [0]:
all_words_neg = sentim_analyzer.all_words([nltk.sentiment.util.mark_negation(doc) for doc in training_docs])

In [0]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
sentim_analyzer.add_feat_extractor(nltk.sentiment.util.extract_unigram_feats, unigrams=unigram_feats)

In [0]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(test_docs)

In [0]:
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
     print('{0}: {1}'.format(key, value))

Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.9802690582959641
F-measure [ham]: 0.9885297184567259
F-measure [spam]: 0.9294871794871795
Precision [ham]: 0.9823834196891191
Precision [spam]: 0.9666666666666667
Recall [ham]: 0.9947534102833159
Recall [spam]: 0.8950617283950617
